In [16]:
import sys
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
sys.path.append("/home/weber/PycharmProjects/EXOTIC/src")
from utils.utils import load_config_file, mkdir

yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/clean/src/config_clean_clean.yaml")


### Fct to compute Odds Ratio

In [25]:
def stats(cat_const, cat_alt, others_const, others_alt, col, cat, ratio=None):
    
#     print(others_alt)
#     odd_ratio = (cat_const/others_const) / (cat_alt/others_alt)
    odd_ratio, pvalue = fisher_exact([[cat_const, cat_alt], [others_const, others_alt]])

    se = np.sqrt((1/cat_const) + (1/others_const) + (1/cat_alt) + (1/others_alt))
    lower = np.exp(np.log(odd_ratio) - 1.96 *se)
    upper = np.exp(np.log(odd_ratio) + 1.96 *se)
#     print(odd_ratio, se, lower, upper)
    print(col, ratio, cat, cat_const, cat_alt, others_const, others_alt, odd_ratio, se, lower, upper)

    convert_col = {
        'AF_bins' : 'MAF',
        'most_severe_consequence' : 'Consequence',
        'CCR_bins' : 'Constraint',
        'phyloCSF_cat' : 'Conservation',
        'Status' : 'Status',
    }
    
    d = {
            'group' : convert_col[col],
            'Cat' : cat,
            'ratio' : ratio,
            'OR' : odd_ratio,
            'SE' : se,
            'IC_low' : lower,
            'IC_up' : upper,
            'pvalue' : pvalue,
            'Const OK' : cat_const,
            'Alt OK' : cat_alt,
            'Const_NOT OK' : others_const,
            'Alt_NOT OK' : others_alt,
        }
    return d

def compute_odds_ratio(df, cols):
    l = list()
    for col in cols:
#         print(col)
#         print(df[['Const_Alt', col]].pivot(columns='Const_Alt', values=col).apply(pd.Series.value_counts))
        t = df[['Const_Alt', col]].pivot(columns='Const_Alt', values=col).apply(pd.Series.value_counts).fillna(0)
#         print(t)
        for i, cat in enumerate(df[col].unique().tolist()):
#             print("Cat : ", cat)
            cat_const = t.loc[cat]['Const']
            cat_alt = t.loc[cat]['Alt']
            others_const = t.loc[~t.index.isin([cat])]['Const'].sum()
            others_alt = t.loc[~t.index.isin([cat])]['Alt'].sum()
            d_stats = stats(cat_const, cat_alt, others_const, others_alt, col, cat, )
            d_stats['id'] = i+1
            l.append(d_stats)
    return l


def compute_odds_ratio_alt(df, cols):
    l = list()
    for col in cols:
        t = df[['Ratio_num_bins_update', col]].dropna(subset=['Ratio_num_bins_update']).pivot(columns='Ratio_num_bins_update', values=col).apply(pd.Series.value_counts).fillna(0)
        t = t[t.columns.dropna()]
#         print(df['Ratio_num_bins_update'].unique().tolist())
        for ratio in sorted(df['Ratio_num_bins_update'].unique().tolist()):
            others_ratio = [c for c in df['Ratio_num_bins_update'].unique().tolist() if c != ratio]
            for i, cat in enumerate(df[col].unique().tolist()):
                freq_ratio_cat_value = t.loc[cat][ratio]
                others_freq_cat_sum = t.loc[cat][others_ratio].sum()
                freq_ratio_others_cat = t.loc[~t.index.isin([cat])][ratio].sum()
                others_freq_others_cat = t.loc[~t.index.isin([cat])][others_ratio].sum(axis=1).values[0]

                d_stats = stats(freq_ratio_cat_value, others_freq_cat_sum, freq_ratio_others_cat, others_freq_others_cat, col, cat, ratio)
                d_stats['id'] = i+1
                l.append(d_stats)
    return l
 

## RefSeq Miso exons

In [3]:
refseq_exons = pd.read_parquet(yaml['2_EXPRESSION']['Final']['refseq_corrected_cds_recomputed'])
refseq_exons.loc[(refseq_exons['Const_Alt'] == 'Alt') & (refseq_exons['Ratio_num_bins'] == '0.8 - 1'), 'Ratio_num_bins_update'] = '0.8 - 0.99'
refseq_exons.loc[(refseq_exons['Const_Alt'] == 'Alt') & (refseq_exons['Ratio_num_bins'] != '0.8 - 1'), 'Ratio_num_bins_update'] = refseq_exons.loc[(refseq_exons['Const_Alt'] == 'Alt') & (refseq_exons['Ratio_num_bins'] != '0.8 - 1')]['Ratio_num_bins']
refseq_exons.loc[(refseq_exons['Const_Alt'] == 'Const') & (refseq_exons['Ratio_num_bins'] == '0.8 - 1'), 'Ratio_num_bins_update'] = '1.00'
refseq_exons['MAP'] = refseq_exons['Gene'] + '_' + refseq_exons['ranges']
refseq_exons

,Gene,ranges,mRNA_exons,mRNA_nb,Share,Strand,mRNA_gene,mRNA_nb_total,Ratio,Ratio_num,Const_Alt,Ratio_num_bins,Ratio_num_bins_update,MAP,Start,End,Length,CDS_count
0,A2M,9220419-9220435,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,False,0,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,1.00,A2M_9220419-9220435,9220419,9220435,16,38
1,A2M,9220779-9220820,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,False,0,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,1.00,A2M_9220779-9220820,9220779,9220820,41,38
2,A2M,9221336-9221438,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,False,0,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,1.00,A2M_9221336-9221438,9221336,9221438,102,38
3,A2M,9222341-9222409,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,False,0,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,1.00,A2M_9222341-9222409,9222341,9222409,68,38
4,A2M,9223084-9223174,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,False,0,"[NM_001347425, NM_001347423, NM_000014, NM_001...",4,4/4,1.0,Const,0.8 - 1,1.00,A2M_9223084-9223174,9223084,9223174,90,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178476,ZZZ3,78047461-78047576,"[NM_001308237, NM_015534]",2,False,0,"[NM_001308237, NM_015534]",2,2/2,1.0,Const,0.8 - 1,1.00,ZZZ3_78047461-78047576,78047461,78047576,115,12
178477,ZZZ3,78047664-78047811,"[NM_001308237, NM_015534]",2,False,0,"[NM_001308237, NM_015534]",2,2/2,1.0,Const,0.8 - 1,1.00,ZZZ3_78047664-78047811,78047664,78047811,147,12
178478,ZZZ3,78050202-78050340,"[NM_001308237, NM_015534]",2,False,0,"[NM_001308237, NM_015534]",2,2/2,1.0,Const,0.8 - 1,1.00,ZZZ3_78050202-78050340,78050202,78050340,138,12
178479,ZZZ3,78097535-78099039,[NM_015534],1,False,0,"[NM_001308237, NM_015534]",2,1/2,0.5,Alt,0.4 - 0.6,0.4 - 0.6,ZZZ3_78097535-78099039,78097535,78099039,1504,12


## PhyloCSF

In [4]:
phylocsf_refseq = pd.read_parquet(yaml['3_EXONS_PROPERTIES']['Final']['refseq_phylocsf'])
phylocsf_refseq = phylocsf_refseq.rename({'target' : 'phyloCSF_score'}, axis=1)
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] <= -100), 'phyloCSF_cat'] = 'Unconserved'
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] > 1000), 'phyloCSF_cat'] = 'Conserved'
phylocsf_refseq.loc[(phylocsf_refseq['phyloCSF_score'] >= -100) & (phylocsf_refseq['phyloCSF_score'] < 1000), 'phyloCSF_cat'] = 'Standard'
phylocsf_refseq = phylocsf_refseq.fillna(False)
phylocsf_refseq = phylocsf_refseq.loc[phylocsf_refseq['phyloCSF_cat'] != 'Standard']
phylocsf_refseq

,interval,phyloCSF_score,start,end,MAP,Gene,phyloCSF_cat
57660,[10:101089145-101090717),5643.7434,10:101089145,10:101090717,CNNM1_101089145-101090717,CNNM1,Conserved
57662,[10:101089691-101090717),3729.0848,10:101089691,10:101090717,CNNM1_101089145-101090717,CNNM1,Conserved
57663,[10:101090240-101090717),2077.2201,10:101090240,10:101090717,CNNM1_101089145-101090717,CNNM1,Conserved
57682,[10:101458291-101458615),1293.5257,10:101458291,10:101458615,ENTPD7_101458291-101458615,ENTPD7,Conserved
57694,[10:101639568-101640118),1125.4920,10:101639568,10:101640118,DNMBP_101639568-101640118,DNMBP,Conserved
...,...,...,...,...,...,...,...
110044,[Y:16734000-16734471),1757.4552,Y:16734000,Y:16734471,NLGN4Y_16734000-16734471,NLGN4Y,Conserved
110045,[Y:16734061-16734471),1652.5963,Y:16734061,Y:16734471,NLGN4Y_16734000-16734471,NLGN4Y,Conserved
110049,[Y:16941610-16942399),3028.8020,Y:16941610,Y:16942399,NLGN4Y_16941610-16942399,NLGN4Y,Conserved
109991,[Y:2829115-2829687),1279.9484,Y:2829115,Y:2829687,ZFY_2829115-2829687,ZFY,Conserved


## gnomAD

In [6]:
gnomad = pd.read_parquet(yaml['3_EXONS_PROPERTIES']['Final']['refseq_gnomad_hail_retrieved'])
"{:e}".format(0.001)
bins_af = list(reversed([1/(10 ** e) for e in list(range(0,7,1))]))

def convert_bins_into_labels(bins):
#     return [" - ".join([str(bins[j]), str(bins[j+1])]) for j, e in enumerate(bins) if j <len(bins)-1]
    return [" - ".join(["{:.0e}".format(bins[j]), "{:.0e}".format(bins[j+1])]) for j, e in enumerate(bins) if j <len(bins)-1]



gnomad['AF'] = gnomad['AF'].str.replace(',', '.').astype(float)
gnomad['AF_bins'] = pd.cut(gnomad['AF'], bins=bins_af, labels=convert_bins_into_labels(bins_af))
mc = ["missense_variant", "start_lost", "stop_gained", "stop_lost", "stop_retained_variant", "synonymous_variant", ]
gnomad = gnomad.loc[gnomad['most_severe_consequence'].isin(mc)]
gnomad

,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,MAP,Gene,AF_bins
0,1:901913,"[""T"",""G""]",0.000012,rs759148200,snv,901913,1_901913_T_G,start_lost,PLEKHN1_901912-901994,PLEKHN1,1e-05 - 1e-04
1,1:901916,"[""G"",""A""]",0.000004,rs764917347,snv,901916,1_901916_G_A,missense_variant,PLEKHN1_901912-901994,PLEKHN1,1e-06 - 1e-05
2,1:901917,"[""G"",""A""]",0.000004,rs774961129,snv,901917,1_901917_G_A,synonymous_variant,PLEKHN1_901912-901994,PLEKHN1,1e-06 - 1e-05
3,1:901919,"[""A"",""C""]",0.000226,rs763342668,snv,901919,1_901919_A_C,missense_variant,PLEKHN1_901912-901994,PLEKHN1,1e-04 - 1e-03
4,1:901920,"[""C"",""G""]",0.000004,rs1408404600,snv,901920,1_901920_C_G,missense_variant,PLEKHN1_901912-901994,PLEKHN1,1e-06 - 1e-05
...,...,...,...,...,...,...,...,...,...,...,...
2810848,Y:22749975,"[""C"",""T""]",0.000017,rs746588234,snv,22749975,Y_22749975_C_T,synonymous_variant,EIF1AY_22749910-22749991,EIF1AY,1e-05 - 1e-04
2810849,Y:22751395,"[""T"",""C""]",0.000015,rs1382594808,snv,22751395,Y_22751395_T_C,synonymous_variant,EIF1AY_22751370-22751461,EIF1AY,1e-05 - 1e-04
2810850,Y:22751399,"[""C"",""G""]",0.000015,rs762602059,snv,22751399,Y_22751399_C_G,missense_variant,EIF1AY_22751370-22751461,EIF1AY,1e-05 - 1e-04
2810851,Y:22751428,"[""C"",""T""]",0.000030,rs1158365540,snv,22751428,Y_22751428_C_T,synonymous_variant,EIF1AY_22751370-22751461,EIF1AY,1e-05 - 1e-04


## ClinVar

In [10]:
clinvar = pd.read_parquet(yaml['3_EXONS_PROPERTIES']['Final']['refseq_clinvar_hail_retrieved'])
clinvar['id'] = clinvar['locus.contig'] + '_' + clinvar['locus.position'].astype(str) + '_' + clinvar['alleles'].apply(lambda r: "_".join(r))
clinvar

,locus.contig,locus.position,alleles,rsid,ALLELEID,CLNREVSTAT,CLNSIG,GENEINFO,RS,MAP,Gene,RS_STARS,snpId,id
71,1,1289293,"[A, T]",402162,389106,"criteria_provided,_single_submitter",Likely_pathogenic,MXRA8:54587,[374879755],MXRA8_1289228-1289308,MXRA8,1,1_1289293_A_T,1_1289293_A_T
72,1,1372327,"[C, T]",830326,818588,no_assertion_criteria_provided,Likely_pathogenic,VWA1:64856,None,VWA1_1372307-1372864,VWA1,0,1_1372327_C_T,1_1372327_C_T
82,1,1451416,"[T, G]",828051,816386,no_assertion_criteria_provided,Pathogenic,ATAD3A:55210,[1570319915],ATAD3A_1451392-1451468,ATAD3A,0,1_1451416_T_G,1_1451416_T_G
89,1,1454346,"[C, T]",828050,816385,no_assertion_criteria_provided,Pathogenic,ATAD3A:55210,[760826883],ATAD3A_1454301-1454370,ATAD3A,0,1_1454346_C_T,1_1454346_C_T
101,1,1460622,"[T, G]",637018,624841,"criteria_provided,_single_submitter",Pathogenic,ATAD3A:55210,[1570345942],ATAD3A_1460620-1460671,ATAD3A,1,1_1460622_T_G,1_1460622_T_G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256896,X,154250707,"[C, A]",10156,25195,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852379],F8_154250685-154250827,F8,0,X_154250707_C_A,X_154250707_C_A
256897,X,154250724,"[T, C]",10352,25391,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852476],F8_154250685-154250827,F8,0,X_154250724_T_C,X_154250724_T_C
256899,X,154250739,"[T, A]",10154,25193,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852378],F8_154250685-154250827,F8,0,X_154250739_T_A,X_154250739_T_A
256900,X,154250751,"[A, C]",10153,25192,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852377],F8_154250685-154250827,F8,0,X_154250751_A_C,X_154250751_A_C


## CCR data

In [8]:
ccr = pd.read_parquet(yaml['3_EXONS_PROPERTIES']['Final']['refseq_ccr_start_end'])
bins = [0,20,80,90,95,100]
ccr['CCR_bins'] = pd.cut(ccr['target'], bins, labels=[" - ".join([str(bins[j]), str(bins[j+1])]) for j, e in enumerate(bins) if j <len(bins)-1], include_lowest=True)
ccr

,interval,target,start,end,MAP,Gene,CCR_bins
1953387,[10:100003848-100003856),63.055177,10:100003848,10:100003856,R3HCC1L_100003848-100003915,R3HCC1L,20 - 80
1953388,[10:100003856-100003857),0.000000,10:100003856,10:100003857,R3HCC1L_100003848-100003915,R3HCC1L,0 - 20
1907621,[10:100003857-100003858),0.000000,10:100003857,10:100003858,R3HCC1L_100003848-100003915,R3HCC1L,0 - 20
1953390,[10:100003858-100003874),71.921144,10:100003858,10:100003874,R3HCC1L_100003848-100003915,R3HCC1L,20 - 80
1907623,[10:100003874-100003875),0.000000,10:100003874,10:100003875,R3HCC1L_100003848-100003915,R3HCC1L,0 - 20
...,...,...,...,...,...,...,...
3526305,[X:99956953-99956958),8.932019,X:99956953,X:99956958,SYTL4_99956924-99957033,SYTL4,0 - 20
3526306,[X:99956958-99956959),0.000000,X:99956958,X:99956959,SYTL4_99956924-99957033,SYTL4,0 - 20
3526307,[X:99956959-99957026),90.007142,X:99956959,X:99957026,SYTL4_99956924-99957033,SYTL4,90 - 95
3611490,[X:99957026-99957027),0.000000,X:99957026,X:99957027,SYTL4_99956924-99957033,SYTL4,0 - 20


## Merge gnomAD & ClinVar

In [11]:
merge_variants = pd.concat(
    [
        gnomad.loc[~gnomad['id'].isin(clinvar.id.values.tolist())],
        clinvar.loc[~clinvar['id'].isin(gnomad.id.values.tolist())]
    ]
)
merge_variants.loc[merge_variants['CLNSIG'].isna() == True, 'Status'] = 'Benign'
merge_variants['Status'] = merge_variants['Status'].fillna('Pathogenic')
merge_variants

,locus,alleles,AF,rsid,variant_type,end,id,most_severe_consequence,MAP,Gene,...,locus.contig,locus.position,ALLELEID,CLNREVSTAT,CLNSIG,GENEINFO,RS,RS_STARS,snpId,Status
0,1:901913,"[""T"",""G""]",0.000012,rs759148200,snv,901913.0,1_901913_T_G,start_lost,PLEKHN1_901912-901994,PLEKHN1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benign
1,1:901916,"[""G"",""A""]",0.000004,rs764917347,snv,901916.0,1_901916_G_A,missense_variant,PLEKHN1_901912-901994,PLEKHN1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benign
2,1:901917,"[""G"",""A""]",0.000004,rs774961129,snv,901917.0,1_901917_G_A,synonymous_variant,PLEKHN1_901912-901994,PLEKHN1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benign
3,1:901919,"[""A"",""C""]",0.000226,rs763342668,snv,901919.0,1_901919_A_C,missense_variant,PLEKHN1_901912-901994,PLEKHN1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benign
4,1:901920,"[""C"",""G""]",0.000004,rs1408404600,snv,901920.0,1_901920_C_G,missense_variant,PLEKHN1_901912-901994,PLEKHN1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256896,NaN,"[C, A]",NaN,10156,NaN,NaN,X_154250707_C_A,NaN,F8_154250685-154250827,F8,...,X,154250707.0,25195.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852379],0.0,X_154250707_C_A,Pathogenic
256897,NaN,"[T, C]",NaN,10352,NaN,NaN,X_154250724_T_C,NaN,F8_154250685-154250827,F8,...,X,154250724.0,25391.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852476],0.0,X_154250724_T_C,Pathogenic
256899,NaN,"[T, A]",NaN,10154,NaN,NaN,X_154250739_T_A,NaN,F8_154250685-154250827,F8,...,X,154250739.0,25193.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852378],0.0,X_154250739_T_A,Pathogenic
256900,NaN,"[A, C]",NaN,10153,NaN,NaN,X_154250751_A_C,NaN,F8_154250685-154250827,F8,...,X,154250751.0,25192.0,no_assertion_criteria_provided,Pathogenic,F8:2157,[137852377],0.0,X_154250751_A_C,Pathogenic


# Compute OR for Const VS Alt

In [17]:
# TMP TO LOAD CONFIG
yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/clean/src/config_clean_clean.yaml")

complete_list_stats = list()

list_df = [phylocsf_refseq]
list_cols = [['phyloCSF_cat']]

for df, cols in zip(list_df, list_cols):
    tmp_m = pd.merge(refseq_exons.dropna(subset=['Ratio_num_bins_update']), df, on='MAP')
    complete_list_stats.extend(compute_odds_ratio(tmp_m, cols))
    
df_or_stat = pd.DataFrame(complete_list_stats)
df_or_stat['Cat'] = df_or_stat['Cat'].str.replace('_' , ' ')
# df_or_stat.to_excel(yaml['3_EXONS_PROPERTIES']['Figures_data']['odds_ratio_const_alt'])
df_or_stat

,group,Cat,ratio,OR,SE,IC_low,IC_up,pvalue,Const OK,Alt OK,Const_NOT OK,Alt_NOT OK,id
0,Conservation,Unconserved,None,0.350292,0.043888,0.321419,0.381758,4.604263e-128,1742,1797,4735,1711,1
1,Conservation,Conserved,None,2.854762,0.043888,2.619458,3.111203,4.604263e-128,4735,1711,1742,1797,2


In [ ]:
# TMP TO LOAD CONFIG
yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/clean/src/config_clean_clean.yaml")

complete_list_stats = list()

list_df = [phylocsf_refseq, gnomad, merge_variants, ccr]
list_cols = [['phyloCSF_cat'], ['most_severe_consequence', 'AF_bins'], ['Status'], ['CCR_bins']]

for df, cols in zip(list_df, list_cols):
    tmp_m = pd.merge(refseq_exons.dropna(subset=['Ratio_num_bins_update']), df, on='MAP')
    complete_list_stats.extend(compute_odds_ratio(tmp_m, cols))
    
df_or_stat = pd.DataFrame(complete_list_stats)
df_or_stat['Cat'] = df_or_stat['Cat'].str.replace('_' , ' ')
# df_or_stat.to_excel(yaml['3_EXONS_PROPERTIES']['Figures_data']['odds_ratio_const_alt'])
df_or_stat

In [29]:
tmp = pd.read_excel(yaml['3_EXONS_PROPERTIES']['Figures_data']['odds_ratio_const_alt'])
tmp = tmp.sort_values(by=['group', 'Cat'])
tmp[['OR', 'SE', 'IC_low', 'IC_up']] = tmp[['OR', 'SE', 'IC_low', 'IC_up']].round(2)
tmp

,Unnamed: 0,group,Cat,ratio,OR,SE,IC_low,IC_up,pvalue,Const OK,Alt OK,Const_NOT OK,Alt_NOT OK,id
3,3,Consequence,missense variant,NaN,0.95,0.00,0.94,0.95,4.840154e-83,1297022,539512,669929,263960,2
6,6,Consequence,start lost,NaN,0.40,0.03,0.38,0.42,2.826548e-284,3105,3187,1963846,800285,5
4,4,Consequence,stop gained,NaN,0.86,0.01,0.85,0.88,1.155900e-51,34791,16399,1932160,787073,3
5,5,Consequence,stop lost,NaN,0.54,0.04,0.51,0.58,1.626188e-61,1802,1352,1965149,802120,4
7,7,Consequence,stop retained variant,NaN,0.59,0.06,0.52,0.67,2.420526e-16,606,421,1966345,803051,6
2,2,Consequence,synonymous variant,NaN,1.09,0.00,1.08,1.09,1.520289e-192,629625,242601,1337326,560871,1
1,1,Conservation,Conserved,NaN,2.85,0.04,2.62,3.11,4.604263e-128,4735,1711,1742,1797,2
0,0,Conservation,Unconserved,NaN,0.35,0.04,0.32,0.38,4.604263e-128,1742,1797,4735,1711,1
17,17,Constraint,0 - 20,NaN,0.92,0.00,0.92,0.93,5.981530e-192,1981071,825930,604224,232163,2
18,18,Constraint,20 - 80,NaN,1.05,0.00,1.05,1.06,2.126345e-75,530950,208300,2054345,849793,3


# Compute OR for exonic gradient frequency

In [25]:
# TMP TO LOAD CONFIG
yaml = load_config_file(config_file="/home/weber/PycharmProjects/EXOTIC/clean/src/config_clean_clean.yaml")

complete_list_stats = list()

list_df = [phylocsf_refseq, gnomad, merge_variants, ccr]
list_cols = [['phyloCSF_cat'], ['most_severe_consequence', 'AF_bins'], ['Status'], ['CCR_bins']]

for df, cols in zip(list_df, list_cols):
    tmp_m = pd.merge(refseq_exons.dropna(subset=['Ratio_num_bins_update']), df, on='MAP')
    complete_list_stats.extend(compute_odds_ratio_alt(tmp_m, cols))
    
df_or_stat = pd.DataFrame(complete_list_stats)
df_or_stat['Cat'] = df_or_stat['Cat'].str.replace('_' , ' ')
df_or_stat.to_excel(yaml['3_EXONS_PROPERTIES']['Figures_data']['odds_ratio_const_alt_gradient'])
df_or_stat

,group,Cat,ratio,OR,SE,IC_low,IC_up,pvalue,Const OK,Alt OK,Const_NOT OK,Alt_NOT OK,id
0,Conservation,Unconserved,0 - 0.2,5.646330,0.124058,4.427575,7.200565,6.157649e-52,262,3277,90,6356,1
1,Conservation,Conserved,0 - 0.2,0.177106,0.124058,0.138878,0.225857,6.157649e-52,90,6356,262,3277,2
2,Conservation,Unconserved,0.2 - 0.4,3.205327,0.077339,2.754484,3.729963,9.444457e-53,473,3066,296,6150,1
3,Conservation,Conserved,0.2 - 0.4,0.311981,0.077339,0.268099,0.363044,9.444457e-53,296,6150,473,3066,2
4,Conservation,Unconserved,0.4 - 0.6,1.865827,0.059558,1.660253,2.096856,2.556626e-25,640,2899,682,5764,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Constraint,90 - 95,1.00,1.034403,0.015551,1.003350,1.066417,2.972012e-02,17549,5457,2567746,825930,1
122,Constraint,0 - 20,1.00,0.826891,0.002859,0.822271,0.831537,0.000000e+00,1981071,825930,604224,208300,2
123,Constraint,20 - 80,1.00,1.024788,0.002895,1.018990,1.030620,2.534699e-17,530950,208300,2054345,825930,3
124,Constraint,95 - 100,1.00,1.132520,0.018849,1.091444,1.175142,2.611431e-11,12801,3629,2572494,825930,4


In [31]:
tmp = pd.read_excel(yaml['3_EXONS_PROPERTIES']['Figures_data']['odds_ratio_const_alt_gradient'])
tmp = tmp.sort_values(by=['group', 'Cat', 'ratio.1'])
tmp[['OR', 'SE', 'IC_low', 'IC_up']] = tmp[['OR', 'SE', 'IC_low', 'IC_up']].round(2)
tmp

,Unnamed: 0,group,Cat,ratio,OR,SE,IC_low,IC_up,pvalue,Const OK,Alt OK,Const_NOT OK,Alt_NOT OK,id
13,13,Consequence,missense variant,0 - 0.2,1.01,0.01,0.99,1.03,4.080406e-01,29559,1806975,13942,859678,2
19,19,Consequence,missense variant,0.2 - 0.4,0.98,0.01,0.97,0.99,2.264742e-03,94338,1742196,45795,830742,2
25,25,Consequence,missense variant,0.4 - 0.6,0.97,0.00,0.96,0.98,9.953412e-14,222004,1614530,109259,771567,2
31,31,Consequence,missense variant,0.6 - 0.8,0.97,0.00,0.96,0.98,1.760307e-10,158513,1678021,77832,800150,2
37,37,Consequence,missense variant,0.8 - 0.99,0.97,0.01,0.96,0.99,5.061799e-03,35098,1801436,17132,856392,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,87,Status,Pathogenic,0.2 - 0.4,0.54,0.04,0.50,0.59,6.384516e-67,689,23732,139975,2625364,2
89,89,Status,Pathogenic,0.4 - 0.6,0.91,0.02,0.88,0.95,6.249360e-06,2693,21728,330748,2434591,2
91,91,Status,Pathogenic,0.6 - 0.8,1.26,0.02,1.21,1.31,4.207859e-27,2571,21850,235839,2529500,2
93,93,Status,Pathogenic,0.8 - 0.99,0.84,0.05,0.76,0.93,6.603780e-04,389,24032,52170,2713169,2
